In [1]:
import pandas as pd
import os
import numpy as np
import itertools
from sklearn.ensemble import RandomForestClassifier
from scipy.spatial.distance import euclidean
import joblib

pd.set_option('display.max_columns', None)

In [2]:
dsdir = 'dataset/coupon-purchase-prediction'

In [3]:
train = pd.read_csv('CPP_REPRO_cl_train.csv').sample(frac=1, random_state=0).reset_index(drop=True)
test =  pd.read_csv('CPP_REPRO_cl_test.csv')
submission = pd.read_csv(os.path.join(dsdir,'sample_submission.csv'))

In [4]:
#train.DISPFROM = pd.to_datetime(train.DISPFROM)
#train.DISPEND = pd.to_datetime(train.DISPEND)
#train.VALIDFROM = pd.to_datetime(train.VALIDFROM)
#train.VALIDEND = pd.to_datetime(train.VALIDEND)
#train.REG_DATE = pd.to_datetime(train.REG_DATE)
#train.WITHDRAW_DATE = pd.to_datetime(train.WITHDRAW_DATE)

#test.DISPFROM = pd.to_datetime(test.DISPFROM)
#test.DISPEND = pd.to_datetime(test.DISPEND)
#test.VALIDFROM = pd.to_datetime(test.VALIDFROM)
#test.VALIDEND = pd.to_datetime(test.VALIDEND)
#test.REG_DATE = pd.to_datetime(test.REG_DATE)
#test.WITHDRAW_DATE = pd.to_datetime(test.WITHDRAW_DATE)

In [5]:
train.drop(['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND', 'REG_DATE', 'WITHDRAW_DATE'], axis=1, inplace=True)
test.drop(['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND', 'REG_DATE', 'WITHDRAW_DATE'], axis=1, inplace=True)

In [6]:
#train.drop(['USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY'], axis=1, inplace=True)
#test.drop(['USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY'], axis=1, inplace=True)

In [7]:
train['SAME_PREF'] = train['PREF_NAME_COUPON'] == train['PREF_NAME_USER']
test['SAME_PREF'] =  test['PREF_NAME_COUPON'] == test['PREF_NAME_USER']

In [8]:
def calc_euc(row):
    return euclidean((row['LATITUDE_COUPON'],row['LONGITUDE_COUPON']),(row['LATITUDE_USER'],row['LONGITUDE_USER']))

In [9]:
print('TRAIN-LONGITUDE_DIST')
train['LONGITUDE_DIST'] = train.apply(lambda x: x.LONGITUDE_COUPON - x.LONGITUDE_USER,axis=1)
print('TRAIN-LATITUDE_DIST')
train['LATITUDE_DIST'] = train.apply(lambda x: x.LATITUDE_COUPON - x.LATITUDE_USER,axis=1)
print('TRAIN-EUCLIDEAN_DIST')
train['EUCLIDEAN_DIST'] = train.apply(calc_euc,axis=1)

test['LONGITUDE_DIST'] = test.apply(lambda x: x.LONGITUDE_COUPON - x.LONGITUDE_USER,axis=1)
print('TEST-LONGITUDE_DIST')
test['LATITUDE_DIST'] = test.apply(lambda x: x.LATITUDE_COUPON - x.LATITUDE_USER,axis=1)
print('TEST-LATITUDE_DIST')
test['EUCLIDEAN_DIST'] = test.apply(calc_euc,axis=1)
print('TEST-EUCLIDEAN_DIST')

TRAIN-LONGITUDE_DIST
TRAIN-LATITUDE_DIST
TRAIN-EUCLIDEAN_DIST
TEST-LONGITUDE_DIST
TEST-LATITUDE_DIST
TEST-EUCLIDEAN_DIST


In [10]:
train.to_csv('CPP_REPRO_cl_train_PLUSPLUS.csv', index=False)
test.to_csv('CPP_REPRO_cl_test_PLUSPLUS.csv', index=False)

In [11]:
x_train = train.drop(['USER_ID_hash', 'COUPON_ID_hash', 'TARGET'],axis=1)
y_train = train.TARGET.values.reshape(-1)

x_test = test.drop(['USER_ID_hash', 'COUPON_ID_hash'],axis=1)

In [12]:
categoricals = x_train.dtypes[x_train.dtypes == 'object'].index.tolist()

In [13]:
categoricals

['CAPSULE_TEXT',
 'GENRE_NAME',
 'LARGE_AREA_NAME',
 'PREF_NAME_COUPON',
 'SMALL_AREA_NAME',
 'SEX_ID',
 'PREF_NAME_USER']

In [14]:
x_train_d = pd.get_dummies(x_train, columns=categoricals)
x_test_d = pd.get_dummies(x_test, columns=categoricals)

x_train_d, x_test_d = x_train_d.align(x_test_d, join='left', axis=1)

x_train_d.fillna(0,inplace=True)
x_test_d.fillna(0,inplace=True)

In [15]:
rfc = RandomForestClassifier(random_state=0, verbose=1, n_jobs=-1, n_estimators=240, class_weight="balanced_subsample")

In [16]:
rfc.fit(x_train_d, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  5.5min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=240, n_jobs=-1, oob_score=False,
                       random_state=0, verbose=1, warm_start=False)

In [17]:
rfc.classes_

array([0, 1], dtype=int64)

In [18]:
joblib.dump(rfc,'CPP_REPO_RF.mdl')

['CPP_REPO_RF.mdl']

In [19]:
%%time
y_pred = rfc.predict_proba(x_test_d)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    5.5s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   29.2s
[Parallel(n_jobs=12)]: Done 240 out of 240 | elapsed:   39.5s finished


Wall time: 5min 54s


In [20]:
sub = test[['USER_ID_hash','COUPON_ID_hash']]
sub['TARGET'] = y_pred[:,1]

C:\Users\Yukha\.conda\envs\tensor\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
grouped = sub.groupby('USER_ID_hash')

In [22]:
def get_top10(row):
    pred = grouped.get_group(row.USER_ID_hash).sort_values(by=['TARGET'],ascending=False)
    pred = ' '.join(map(str, pred.head(10).COUPON_ID_hash.values))
    return pred

In [23]:
submission['PURCHASED_COUPONS'] = submission.apply(get_top10, axis=1)

In [24]:
submission.to_csv('sub_CPP_REPRO_RF.csv', index=False)

In [25]:
submission

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,98dc8da9b3c7ca94aeaa398aef52784f 281326ffac6d5...
1,00035b86e6884589ec8d28fbf2fe7757,f5a77f2907876411752d58e1b9030023 fc5f052a1bd97...
2,0005b1068d5f2b8f2a7c978fcfe1ca06,96d275611439e0029db651a914965ea3 9bfec8781a9c9...
3,000cc06982785a19e2a2fdb40b1c9d59,79de77aa8c36fdf17cb3366e2084e353 0e917a0e87224...
4,0013518e41c416cd6a181d277dd8ca0b,c988d799bc7db9254fe865ee6cf2d4ff 46da51ba6dd20...
...,...,...
22868,fff1a623187cefd7a594e338709b0f40,51da52d5516033bea13972588b671184 42cc500acba3c...
22869,fff4a076cfda6ff9dbe85e1cb678791b,98dc8da9b3c7ca94aeaa398aef52784f 79de77aa8c36f...
22870,fff970d2014c3e10a77e38d540239017,f0685cf6de3c1e1fd86d2f10784b85f5 0acc89ba7593e...
22871,fffafc024e264d5d539813444cf61199,c60dbd64087f40d46d539a96947d0e87 27741884a086e...
